# Text into vector conversion

In [3]:
import pandas as pd

df = pd.read_csv('IMDB_Dataset.csv')
df 


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


Getting the vocabulary from the unique values from the text input

In [111]:
sent_tokens = df.review.map(lambda x: set(x.split(' ')))
vocab = sent_tokens.explode().unique()
print(f'Vocabulary size: {vocab.shape[0]}')

Vocabulary size: 439838


Assiging index values to each unique words in the vocabulary

In [5]:
word_index = {}
for index , row in enumerate(sent_tokens):
    for word in row:
        word_index[word] = index

Vocabulary size: 439838


Assigning index values for each words in the sentences , therby converting text into vector representation

In [6]:
sent_indices = sent_tokens.map(lambda x: [word_index[word] for word in x])
print(f'Indieces len : {len(sent_indices[0])}') #len(sent_indices[0])
print(f'Sentences len : {len(sent_indices[0])}')

Indieces len : 208
Sentences len : 208


Converting postive and negatives to 1 and 0 respectively

In [7]:
target = df.sentiment.map(lambda x: 1 if x == 'positive' else 0)
target.shape

(50000,)

# Model development

Weights Initialization

In [112]:
import numpy as np

hidden_size = 100
alpha, iteration = 0.01, 10

weights_0_1 = 0.2*np.random.random((len(vocab),hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,1)) - 0.1

Neural network to compute the sentiments from the reviews

In [113]:
def sigmoid(x):
    return 1/(1+np.exp(-x))


def neural_network(x,y):

    global weights_0_1, weights_1_2 

    # print(f'x {len(x)}')
    layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
    # print(f'Layer 1 shape: {layer_1.shape}')
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
    
    layer_2_delta = layer_2 - y
    # print(f'weights_1_2 shape: {weights_1_2.T.shape}')

    layer_1_delta = layer_2_delta.dot(weights_1_2.T)  
    # print(f'layer_1_delta shape: {layer_1_delta.shape}')

    weights_1_2 -= np.outer(layer_1 , layer_2_delta) * alpha
    weights_0_1[x] -= layer_1_delta * alpha

    return layer_2

correct , total = 0, 0
for i in range(iteration):
    for x,y in zip(sent_indices,target):
        output = neural_network(x,y)

        correct += 1 if np.abs(output) < 0.5 else 0
        total += 1

    print(f'Iteration: {i}, Accuracy: {correct/total}')
    correct , total = 0, 0


Iteration: 0, Accuracy: 0.51224
Iteration: 1, Accuracy: 0.47096
Iteration: 2, Accuracy: 0.42806
Iteration: 3, Accuracy: 0.3811


/tmp/ipykernel_14001/2337424581.py:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


Iteration: 4, Accuracy: 0.34758
Iteration: 5, Accuracy: 0.3325
Iteration: 6, Accuracy: 0.3257
Iteration: 7, Accuracy: 0.32216
Iteration: 8, Accuracy: 0.32204
Iteration: 9, Accuracy: 0.3203
